In [1]:
import pandas as pd
import requests

In [2]:
def get_character(char_name, char_realm, char_region):
    response = requests.get('https://raider.io/api/v1/characters/profile?region='+
                             str(char_region) + '&realm='+
                             str(char_realm) + '&name='+
                             str(char_name) + '&fields=gear%2Ccovenant%2Cmythic_plus_scores_by_season%3Acurrent%2Cmythic_plus_ranks%2C')
    return response.json()

In [3]:
def get_leaderboard(page):
    response = requests.get('https://raider.io/api/v1/mythic-plus/runs?season=season-sl-1&region=world&dungeon=all&page='+
                            str(page))
    return response.json()

In [ ]:
def get_character_extra(char_name, char_realm, char_region):
    response = requests.get('https://raider.io/characters/'+
                             str(char_region) + '/'+
                             str(char_realm) + '/'+
                             str(char_name))
    return response.json()

In [4]:
df2 = pd.DataFrame(columns=['score','run.dungeon.name','run.mythic_level','run.faction','run.weekly_modifiers','run.roster'])
for page in range(0,5):
    temp = get_leaderboard(page)
    temp = pd.json_normalize(temp, record_path = ['rankings'])
    columns = ['score','run.dungeon.name','run.mythic_level','run.faction','run.weekly_modifiers','run.roster']
    temp = temp[columns]
    df2 = df2.append(temp, ignore_index=True)

In [5]:
characters = {'name': [],'class': [],'race': [],'server': [],'region':[]}
for run in df2['run.roster']:
    for char in run:
        if (char.get('role') == 'tank'):
            characters['name'].append(char.get('character').get('name'))
            characters['class'].append(char.get('character').get('class').get('name'))
            characters['race'].append(char.get('character').get('race').get('name'))
            characters['server'].append(char.get('character').get('realm').get('slug'))
            characters['region'].append(char.get('character').get('region').get('short_name'))
            
temp = pd.DataFrame(characters)
df2['affix_1'] = df2['run.weekly_modifiers'].apply(pd.Series)[0].apply(pd.Series)['name']
df2['affix_2'] = df2['run.weekly_modifiers'].apply(pd.Series)[1].apply(pd.Series)['name']
df2['affix_3'] = df2['run.weekly_modifiers'].apply(pd.Series)[2].apply(pd.Series)['name']
df2['affix_4'] = df2['run.weekly_modifiers'].apply(pd.Series)[3].apply(pd.Series)['name']
df2 = df2.drop(['run.weekly_modifiers','run.roster'],axis=1)
df2 = pd.concat([df2, temp], axis=1)
df2 = df2[ df2['region'] != 'CN']

In [7]:
df = pd.DataFrame(columns=['name', 'race', 'class', 'active_spec_name', 'active_spec_role',
       'gender', 'faction', 'achievement_points', 'honorable_kills',
       'thumbnail_url', 'region', 'realm', 'last_crawled_at', 'profile_url',
       'profile_banner', 'covenant', 'mythic_plus_scores_by_season',
       'mythic_plus_ranks.overall.world', 'mythic_plus_ranks.overall.region',
       'mythic_plus_ranks.overall.realm', 'mythic_plus_ranks.class.world',
       'mythic_plus_ranks.class.region', 'mythic_plus_ranks.class.realm',
       'mythic_plus_ranks.faction_overall.world',
       'mythic_plus_ranks.faction_overall.region',
       'mythic_plus_ranks.faction_overall.realm',
       'mythic_plus_ranks.faction_class.world',
       'mythic_plus_ranks.faction_class.region',
       'mythic_plus_ranks.faction_class.realm', 'mythic_plus_ranks.tank.world',
       'mythic_plus_ranks.tank.region', 'mythic_plus_ranks.tank.realm',
       'mythic_plus_ranks.class_tank.world',
       'mythic_plus_ranks.class_tank.region',
       'mythic_plus_ranks.class_tank.realm',
       'mythic_plus_ranks.faction_tank.world',
       'mythic_plus_ranks.faction_tank.region',
       'mythic_plus_ranks.faction_tank.realm',
       'mythic_plus_ranks.faction_class_tank.world',
       'mythic_plus_ranks.faction_class_tank.region',
       'mythic_plus_ranks.faction_class_tank.realm',
       'mythic_plus_ranks.dps.world', 'mythic_plus_ranks.dps.region',
       'mythic_plus_ranks.dps.realm', 'mythic_plus_ranks.class_dps.world',
       'mythic_plus_ranks.class_dps.region',
       'mythic_plus_ranks.class_dps.realm',
       'mythic_plus_ranks.faction_dps.world',
       'mythic_plus_ranks.faction_dps.region',
       'mythic_plus_ranks.faction_dps.realm',
       'mythic_plus_ranks.faction_class_dps.world',
       'mythic_plus_ranks.faction_class_dps.region',
       'mythic_plus_ranks.faction_class_dps.realm',
       'mythic_plus_ranks.spec_577.world', 'mythic_plus_ranks.spec_577.region',
       'mythic_plus_ranks.spec_577.realm',
       'mythic_plus_ranks.faction_spec_577.world',
       'mythic_plus_ranks.faction_spec_577.region',
       'mythic_plus_ranks.faction_spec_577.realm',
       'mythic_plus_ranks.spec_581.world', 'mythic_plus_ranks.spec_581.region',
       'mythic_plus_ranks.spec_581.realm',
       'mythic_plus_ranks.faction_spec_581.world',
       'mythic_plus_ranks.faction_spec_581.region',
       'mythic_plus_ranks.faction_spec_581.realm', 'gear.item_level_equipped',
       'gear.item_level_total', 'gear.artifact_traits', 'gear.items'])
for i, j in df2.iterrows(): 
    char_name = j['name']
    char_realm = j['server']
    char_region = j['region']
    if j['region'] != 'CN':
        data = get_character(char_name,char_realm,char_region)
        temp = pd.json_normalize(data)
        df = df.append(temp, ignore_index=True)
    else:
        df = df.append(pd.Series([],dtype=pd.StringDtype()), ignore_index=True)

In [8]:
columns = ['name','race','class','active_spec_name','active_spec_role','gender',
               'faction','achievement_points','region','realm','covenant.name','covenant.renown_level','gear.item_level_equipped',
               'gear.items.head.name','gear.items.neck.name','gear.items.shoulder.name','gear.items.back.name','gear.items.chest.name',
               'gear.items.waist.name','gear.items.shirt.name','gear.items.wrist.name','gear.items.hands.name','gear.items.legs.name',
               'gear.items.feet.name','gear.items.finger1.name','gear.items.finger2.name','gear.items.trinket1.name',
               'gear.items.trinket2.name','gear.items.mainhand.name','gear.items.offhand.name','mythic_plus_scores_by_season']
df = df[ columns]
df['score'] = df['mythic_plus_scores_by_season'].apply(pd.Series)[0].apply(pd.Series)['scores'].apply(pd.Series)['tank']
df = df.drop('mythic_plus_scores_by_season',axis=1)
df = df.dropna().drop_duplicates()
df.columns =['name','race','class','active_spec_name', 'active_spec_role','gender','faction', 'achievement_points', 'region','realm','covenant','renown','item_level','helm','neck','shoulder','cloak','chest','waist','shirt','wrist','hands','legs','feet','finger1','finger2','trinket1','trinket2','mainhand','offhand','score']

In [17]:
from bs4 import BeautifulSoup
import urllib.request
import csv
# Raider.io has blocked web-scrapping, using Blizzard instead

In [25]:
char_region='EU'
char_realm='tarren-mill'
char_name = 'Naowhtwo'

urlpage = 'https://worldofwarcraft.com/en-gb/character/'+str(char_region).lower() + '/'+str(char_realm) + '/'+str(char_name)
page = urllib.request.urlopen(urlpage)
soup = BeautifulSoup(page,'html.parser')

In [28]:
table = soup.find('div', attrs={'class': 'Grid-1of2'})

In [30]:
soup

<!DOCTYPE html>
<html lang="en-GB"><head><title>Naowhtwo - Character</title><script>var optimizelyEnabled = false;
try { optimizelyEnabled = JSON.parse("true"); } catch (err) { console.log(err); }
</script><script>var optimizelyLoaded = (function () {

	var OPTIMIZELY_AGENT_LOADED_EVENT = "OptimizelyWebLoaded";
	var OPTIMIZELY_FULLSTACK_DATAFILE_LOADED_EVENT = "OptimizelyFullstackDataFileLoaded";

	function initOptimizely() {
		var agentScript = document.createElement("script");
		agentScript.src = "https://cdn.optimizely.com/js/8520856750.js";
		agentScript.onload = function () {
			optimizelyLoaded = true;
			trigger(OPTIMIZELY_AGENT_LOADED_EVENT);
		};
		document.head.appendChild(agentScript);

		var optimizelySdkKey = "6WXZo8UVWkPQHnS8ss4BQ";
		var optimizelySdkEnabled = false;
		try { optimizelySdkEnabled = JSON.parse("true"); } catch (err) { console.log(err); }
		if (optimizelySdkKey && optimizelySdkEnabled) {
			var dataFileScript = document.createElement("script");
			dataFileS

In [29]:
table

In [32]:
page = "https://eu.api.blizzard.com/profile/wow/character/hellfire/skyl%C3%ADne/soulbinds?namespace=profile-eu&locale=en_GB&access_token=USUn93woTSDPQw6rHMWJsAYFv60yNNqJQr"
#        https://eu.api.blizzard.com/profile/wow/character/hellfire/skyl%C3%ADne?namespace=profile-eu&locale=en_GB&access_token=USUn93woTSDPQw6rHMWJsAYFv60yNNqJQr

response = requests.get(page)
response.json()

{'_links': {'self': {'href': 'https://eu.api.blizzard.com/profile/wow/character/hellfire/skyl%C3%ADne/soulbinds?namespace=profile-eu'}},
 'character': {'key': {'href': 'https://eu.api.blizzard.com/profile/wow/character/hellfire/skyl%C3%ADne?namespace=profile-eu'},
  'name': 'Skylíne',
  'id': 56222403,
  'realm': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/realm/1587?namespace=dynamic-eu'},
   'name': 'Hellfire',
   'id': 1587,
   'slug': 'hellfire'}},
 'chosen_covenant': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/covenant/1?namespace=static-9.0.2_36532-eu'},
  'name': 'Kyrian',
  'id': 1},
 'renown_level': 24,
 'soulbinds': [{'soulbind': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/covenant/soulbind/7?namespace=static-9.0.2_36532-eu'},
    'name': 'Pelagos',
    'id': 7},
   'traits': [{'trait': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/tech-talent/1316?namespace=static-9.0.2_36532-eu'},
      'name': 'Combat Meditation',
      'id': 1316}